In [153]:
import pandas as pd
#importation des fonctions de mon dossier modules
import sys
sys.path.append('modules')
from modules.filters import is_english 

from modules.cleaning import clean_text

import de nos datasets

In [154]:


#hams originaux
df_original_h = pd.read_csv("../datasets/ham_original.csv")

#spams originaux
df_original = pd.read_csv("../datasets/spam_original.csv")

#spams atypiques
df1 = pd.read_csv("../datasets/spam_dataset_1.csv")
df2 = pd.read_csv("../datasets/spam_dataset_2.csv")
df3 = pd.read_csv("../datasets/spam_dataset_3.csv")

Supprimer les collones qui ne sont pas en anglais

In [155]:
#supprimze les collones qui ne sont pas en anglais

df1['is_eng'] = df1['text'].apply(is_english)
df1 = df1[df1['is_eng'] == True]

df2['is_eng'] = df2['text'].apply(is_english)
df2 = df2[df2['is_eng'] == True]
    
df3['is_eng'] = df3['text'].apply(is_english)
df3 = df3[df3['is_eng'] == True]

Remplacer les liens, mails et telephone par des tocken

In [156]:
import re

def replace_splitted_urls(text: str) -> str:
    """
    Recherche dans chaque ligne les fragments de la forme :
      - http : / / (ou https : / /)
      - www .
    puis capture tout le reste jusqu'à la fin de la ligne.
    Ensuite, on recolle les espaces et si on obtient bien un "http://", "https://" ou "www.",
    on remplace par le token [URL].
    """

    # (?:https?\s*:\s*/\s*/)   = "http : //..." ou "https : //..." (tolérant des espaces)
    # (?:www\s*\.)            = "www." (tolérant des espaces avant le point)
    #
    # [^\n]*                  = on capture tout le reste jusqu'à la fin de la ligne
    pattern = re.compile(r'((?:https?\s*:\s*/\s*/)|(?:www\s*\.))[^\n]*', flags=re.IGNORECASE)

    def replace_match(m: re.Match) -> str:
        # Partie de texte capturée qui ressemble à une URL fragmentée
        splitted_url = m.group(0)
        # On retire tous les espaces au sein de cette portion
        cleaned_url = re.sub(r'\s+', '', splitted_url)
        
        # Vérif basique : est-ce que le résultat commence par "http://", "https://" ou "www." ?
        if re.match(r'^(?:https?://|www\.)', cleaned_url, flags=re.IGNORECASE):
            return '[ENTITY]'
        else:
            # Sinon, on peut laisser tel quel ou avoir une autre logique
            return splitted_url

    return pattern.sub(replace_match, text)




def replace_splitted_emails(text: str) -> str:
    """
    Détecte les adresses e-mail fragmentées (ex: ab _ karol @ fastermail . com),
    recolle les morceaux, puis remplace par [EMAIL].
    """

    # Pattern qui capture tout ce qui ressemble à : 
    #   <suite de caractères> @ <suite de caractères> . <extension>
    # en tolérant des espaces
    #
    # [\w.\-_]+ correspond à un ensemble de caractères usuels pour la partie locale (avant le @)
    # \s*@\s* tolère les espaces autour du @
    # [\w.\-]+ tolère lettres, chiffres, point, tiret pour le domaine
    # \s*\.\s* tolère les espaces autour du point
    # [a-zA-Z]{2,} => extension de 2 lettres ou plus (com, fr, net, co.uk plus compliqué, etc.)
    pattern = re.compile(r'([\w.\-_]+\s*@\s*[\w.\-]+\s*\.\s*[a-zA-Z]{2,})', flags=re.IGNORECASE)

    def replace_match(m: re.Match) -> str:
        splitted_email = m.group(1)
        # Retirer tous les espaces intérieurs
        cleaned_email = re.sub(r'\s+', '', splitted_email)
        # Vérif basique : est-ce que ça ressemble à un email standard ?
        # Par ex: "ab_karol@fastermail.com"
        if re.match(r'^[\w.\-_]+@[\w.\-]+\.[a-zA-Z]{2,}$', cleaned_email, flags=re.IGNORECASE):
            return '[ENTITY]'
        else:
            return splitted_email  # ou remplacer inconditionnellement par [EMAIL]

    return pattern.sub(replace_match, text)










import re

def replace_phone_numbers(text: str) -> str:
    """
    Remplace des numéros de téléphone (y compris courts) par [PHONE].
    Exemples détectés : +33 6 12 34 56 78, (202) 555-0147, 88039, etc.
    """

    # Ajustement du pattern pour inclure des numéros courts
    # 1. Capture des numéros avec ou sans "+"
    # 2. Accepte des longueurs de 5 à 16 chiffres
    # 3. Tolère des espaces, tirets, parenthèses, points
    phone_pattern = re.compile(r'(\+?\s*\d[\d\-\(\)\s\.]{3,}\d)')

    def phone_replacer(m: re.Match) -> str:
        raw_phone = m.group(1)
        # Enlever espaces, tirets, parenthèses, points
        cleaned_phone = re.sub(r'[\s\-\(\)\.]+', '', raw_phone)

        # Vérification stricte : 5 à 16 chiffres, avec ou sans "+"
        if re.match(r'^\+?\d{5,16}$', cleaned_phone):
            return '[ENTITY]'
        else:
            return raw_phone  # Ne pas remplacer si ça ne ressemble pas à un numéro

    return phone_pattern.sub(phone_replacer, text)

In [157]:

#remplacer les urls par le tocken [URL]
df1['text'] = df1['text'].apply(replace_splitted_urls)
df2['text'] = df2['text'].apply(replace_splitted_urls)
df3['text'] = df3['text'].apply(replace_splitted_urls)  
df_original['text'] = df_original['text'].apply(replace_splitted_urls)
df_original_h['text'] = df_original_h['text'].apply(replace_splitted_emails)

#remplacer les emails par le tocken [EMAIL]
df1['text'] = df1['text'].apply(replace_splitted_emails) 
df2['text'] = df2['text'].apply(replace_splitted_emails)
df3['text'] = df3['text'].apply(replace_splitted_emails)
df_original['text'] = df_original['text'].apply(replace_splitted_emails)
df_original_h['text'] = df_original_h['text'].apply(replace_splitted_emails)

#remplacer les numéros de téléphone par le tocken [PHONE]
df1['text'] = df1['text'].apply(replace_phone_numbers)
df2['text'] = df2['text'].apply(replace_phone_numbers)
df3['text'] = df3['text'].apply(replace_phone_numbers)
df_original['text'] = df_original['text'].apply(replace_phone_numbers)
df_original_h['text'] = df_original_h['text'].apply(replace_phone_numbers)


remplacer les mots explicit par un token

In [158]:
with open("../mesdf/profanities.txt", "r") as file:
    profanities = file.read().splitlines()




# Exemple de filtre avec une liste dynamique
import re

def censor_text_with_explicit(text):
    profanities_regex = re.compile(r'\b(?:' + '|'.join(profanities) + r')\b', re.IGNORECASE)
    return profanities_regex.sub("[explicit]", text)


# Appliquer au DataFrame
#remplacer les numéros de téléphone par le tocken [PHONE]
df1['text'] = df1['text'].apply(censor_text_with_explicit)
df2['text'] = df2['text'].apply(censor_text_with_explicit)
df3['text'] = df3['text'].apply(censor_text_with_explicit)
df_original['text'] = df_original['text'].apply(censor_text_with_explicit)
df_original_h['text'] = df_original_h['text'].apply(censor_text_with_explicit)

df1['text'].head(25).values


array(['abasements darer prudently fortuitous undergone lighthearted charm orinoco taster railroad affluent pornographic cuvier irvin parkhouse blameworthy chlorophyll robed diagrammatic fogarty clears bayda inconveniencing managing represented smartness hashish academies shareholders unload badness danielson pure caffein spaniard chargeable levin ',
       'it is difficult to make our material condition better by the best law , but it is easy enough to ruin it by bad laws . excuse me . . . : ) you just found the best and simpliest site for medication on the net . no perscription , easy delivery . private , secure , and easy . better see rightly on a pound a week than squint on a million . we ` ve got anything that you will ever want . erection treatment pills , anti - depressant pills , weight loss , and more ! [ENTITY]',
       "vcsc - brand new stock for your attention vocalscape inc - the stock symbol is : vcsc vcsc will be our top stock pick for the month of april - stock expected

Creer une colone pour la longueur et standardiser la longeur des spams supplementaires a la meme occasion

In [159]:
#pour les datesets suplémentaires ne garder que les ligne qui ne depassent pas 250 caractères afin quilq soient comme l'original et créer une nouvelle colone pour la longueur en contant les espaces
df1['len'] = df1['text'].apply(len) 
df1 = df1[df1['len'] <= 250]

df2['len'] = df2['text'].apply(len)
df2 = df2[df2['len'] <= 250]

df3['len'] = df3['text'].apply(len)
df3 = df3[df3['len'] <= 250]

df_original['len'] = df_original['text'].apply(len)
df_original_h['len'] = df_original_h['text'].apply(len)

Creer une colone pour la longueur en ne comptant pas les espaces

In [160]:
#cree une nouvelle colone pour la longueur en ne contant pas les espaces
df1['len_no_spaces'] = df1['text'].apply(lambda x: len(x.replace(' ', '')))
df2['len_no_spaces'] = df2['text'].apply(lambda x: len(x.replace(' ', '')))
df3['len_no_spaces'] = df3['text'].apply(lambda x: len(x.replace(' ', '')))
df_original['len_no_spaces'] = df_original['text'].apply(lambda x: len(x.replace(' ', '')))
df_original_h['len_no_spaces'] = df_original_h['text'].apply(lambda x: len(x.replace(' ', '')))


Creer une collone pour le nombre de mots

In [161]:
#nouvelle colone pour le nombre de mots
df1['n_words'] = df1['text'].apply(lambda x: len(x.split()))
df2['n_words'] = df2['text'].apply(lambda x: len(x.split()))
df3['n_words'] = df3['text'].apply(lambda x: len(x.split()))
df_original['n_words'] = df_original['text'].apply(lambda x: len(x.split()))
df_original_h['n_words'] = df_original_h['text'].apply(lambda x: len(x.split()))

Creer une nouvelle colonne pour le nombre de caractère spéciaux sans les espaces

In [162]:
import string
#nouvelle colone pour le nombre de caractères spéciaux en ne comptant pas les espaces
df1['n_special_chars'] = df1['text'].apply(lambda x: sum(c in string.punctuation for c in x)) 
df2['n_special_chars'] = df2['text'].apply(lambda x: sum(c in string.punctuation for c in x))
df3['n_special_chars'] = df3['text'].apply(lambda x: sum(c in string.punctuation for c in x))
df_original['n_special_chars'] = df_original['text'].apply(lambda x: sum(c in string.punctuation for c in x))
df_original_h['n_special_chars'] = df_original_h['text'].apply(lambda x: sum(c in string.punctuation for c in x))


Creer une nouvelle collone pour compter le nombre de majuscule

In [163]:
#compter le nombre de majuscules
df1['n_capital_letters'] = df1['text'].apply(lambda x: sum(1 for c in x if c.isupper()))
df2['n_capital_letters'] = df2['text'].apply(lambda x: sum(1 for c in x if c.isupper()))
df3['n_capital_letters'] = df3['text'].apply(lambda x: sum(1 for c in x if c.isupper()))
df_original['n_capital_letters'] = df_original['text'].apply(lambda x: sum(1 for c in x if c.isupper()))
df_original_h['n_capital_letters'] = df_original_h['text'].apply(lambda x: sum(1 for c in x if c.isupper()))


Mettre en minuscule et supprimer les caractere speciaux

In [164]:
#mettre en minuscule et supprimer les caractères spéciaux
df1['text'] = df1['text'].apply(clean_text)
df2['text'] = df2['text'].apply(clean_text)
df3['text'] = df3['text'].apply(clean_text)
df_original['text'] = df_original['text'].apply(clean_text)
df_original_h['text'] = df_original_h['text'].apply(clean_text)


Creer une nouvelle collone pour compter le nombre de tocken

In [165]:
#Compter le nombre de tokens
def count_emails(text):
    return text.count('[email]')
def count_urls(text):
    return text.count('[url]')
def count_phones(text):
    return text.count('[phone]')
def count_explicit(text):
    return text.count('[explicit]')

df1['emails'] = df1['text'].apply(count_emails)
df1['urls'] = df1['text'].apply(count_urls)
df1['phones'] = df1['text'].apply(count_phones)
df1['explicit'] = df1['text'].apply(count_explicit)

df2['emails'] = df2['text'].apply(count_emails)
df2['urls'] = df2['text'].apply(count_urls)
df2['phones'] = df2['text'].apply(count_phones)
df2['explicit'] = df2['text'].apply(count_explicit)

df3['emails'] = df3['text'].apply(count_emails)
df3['urls'] = df3['text'].apply(count_urls)
df3['phones'] = df3['text'].apply(count_phones)
df3['explicit'] = df3['text'].apply(count_explicit)

df_original['emails'] = df_original['text'].apply(count_emails)
df_original['urls'] = df_original['text'].apply(count_urls)
df_original['phones'] = df_original['text'].apply(count_phones)
df_original['explicit'] = df_original['text'].apply(count_explicit)

df_original_h['emails'] = df_original_h['text'].apply(count_emails)
df_original_h['urls'] = df_original_h['text'].apply(count_urls)
df_original_h['phones'] = df_original_h['text'].apply(count_phones)
df_original_h['explicit'] = df_original_h['text'].apply(count_explicit)

Enregistrement des datasets

In [166]:
#enregistrer les fichiers séparéments
df1.to_csv('../datasets/spam_dataset_1_clean_.csv', index=False)
df2.to_csv('../datasets/spam_dataset_2_clean_.csv', index=False)
df3.to_csv('../datasets/spam_dataset_3_clean_.csv', index=False)
df_original.to_csv('../datasets/spam_clean_.csv', index=False)
df_original_h.to_csv('../datasets/ham_clean_.csv', index=False)

#fusion de mes 3 datasets spam atypiques
df_atypiques = pd.concat([df1, df2, df3], axis=0)
df_atypiques.to_csv('../datasets/spam_atypiques_clean_.csv', index=False)

#fusion de tout les datasets spams
df_all_spams = pd.concat([df_atypiques, df_original], axis=0)
df_all_spams.to_csv('../datasets/all_spams_clean_.csv', index=False)

#fusion de tout!
df_all = pd.concat([df_all_spams, df_original_h], axis=0)
df_all.to_csv('../datasets/all_clean_.csv', index=False)

replace and count tockens

In [167]:
# import re

# def replace_tokens_with_common_token(text, common_token="[entity]"):
#     # Remplacer les tokens spécifiques par un token commun
#     text = re.sub(r'\[email\]', common_token, text)  # Remplacer [mail] par [ENTITY]
#     text = re.sub(r'\[phone\]', common_token, text)  # Remplacer [phone] par [ENTITY]
#     text = re.sub(r'\[url\]', common_token, text)  # Remplacer [url] par [ENTITY]
    
#     return text

# df_all['text_optimized'] = df_all['text'].apply(replace_tokens_with_common_token)
df_all['entities'] = df_all['text'].apply(lambda text: text.count('[entity]'))

Tockeniser

In [168]:
df_all["text"] = df_all["text"].apply(lambda x : x.split())




df_all.tail()

,label,text,is_eng,len,len_no_spaces,n_words,n_special_chars,n_capital_letters,emails,urls,phones,explicit,entities
4511,0,"[huh, y, lei]",NaN,12,10,3,3,1,0,0,0,0,0
4512,0,"[will, b, going, to, esplanade, fr, home]",NaN,37,30,8,2,2,0,0,0,0,0
4513,0,"[pity, was, in, mood, for, that, soany, other,...",NaN,57,48,10,7,2,0,0,0,0,0
4514,0,"[the, guy, did, some, [explicit], but, i, acte...",NaN,127,102,26,3,2,0,0,0,1,0
4515,0,"[rofl, its, true, to, its, name]",NaN,26,21,6,1,2,0,0,0,0,0


Supprimmer les stop words

In [169]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

tout_les_stopword = stopwords.words("english")

def supression_des_stopwords(texte):
    liste = []
    for mot in texte:
        if mot in tout_les_stopword:
            pass
        else:
            liste.append(mot)
    return liste

df_all['text'] = df_all['text'].apply(supression_des_stopwords)





df_all

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,text,is_eng,len,len_no_spaces,n_words,n_special_chars,n_capital_letters,emails,urls,phones,explicit,entities
9,1,"[sends, final, destination, designated, privat...",True,188,150,38,6,0,0,0,0,0,0
17,1,"[emile, cablefilterz, allow, receive, channels...",True,179,148,32,9,6,0,0,0,0,1
20,1,"[muniz, govenment, want, sell, undergroundcd, ...",True,190,157,34,7,6,0,0,0,0,1
22,1,"[tinydrive, 2, 2, gb, 5, x, 2, 25, x, 2, 5, in...",True,169,127,43,17,12,0,0,0,0,2
27,1,"[80, smart, ink, system, [entity]]",True,165,142,24,120,6,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,0,"[huh, lei]",NaN,12,10,3,3,1,0,0,0,0,0
4512,0,"[b, going, esplanade, fr, home]",NaN,37,30,8,2,2,0,0,0,0,0
4513,0,"[pity, mood, soany, suggestions]",NaN,57,48,10,7,2,0,0,0,0,0
4514,0,"[guy, [explicit], acted, like, id, interested,...",NaN,127,102,26,3,2,0,0,0,1,0


Lemmatization

In [170]:
# from nltk import WordNetLemmatizer

# nltk.download('wordnet')

# WordNetLemmatizer().lemmatize("downloading")
# """
# La lemmatisation est une méthode plus précise qui ramène un mot à sa forme canonique, ou à sa racine grammaticale, en prenant en compte le contexte et la signification. Par exemple :

#     "running" -> "run"
#     "better" -> "good"

# """

# def lematisation(liste):
#   lematisation = []
#   for element in liste:
#     elementLematize = WordNetLemmatizer().lemmatize(element)
#     lematisation.append(elementLematize)
#   return lematisation

# df_all['text'] = df_all['text'].apply(lematisation)

# df_all

steeming

In [171]:
from nltk import PorterStemmer
nltk.download('wordnet')
nltk.download('wordnet')

def stemming(liste):
  stemming = []
  for element in liste:
    elementStemme = PorterStemmer().stem(element)
    stemming.append(elementStemme)
  return stemming

df_all['text'] = df_all['text'].apply(stemming)





[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Reccupération des 10 mots les plus présents pour les spams et les hams (extraire des features discriminantes)

In [172]:
from collections import Counter





# Séparer les hams et les spams
hams = df_all[df_all["label"] == 0]['text']
spams = df_all[df_all['label'] == 1]['text']


# Compter les mots directement (les lignes sont déjà tokenisées)
ham_words = Counter(word for tokens in hams for word in tokens)
spam_words = Counter(word for tokens in spams for word in tokens)

# Obtenir les mots les plus fréquents
common_ham_words = [word for word, freq in ham_words.most_common(15)]
common_spam_words = [word for word, freq in spam_words.most_common(15)]

print("Mots fréquents dans les hams globalement :", common_ham_words)
print("Mots fréquents dans les spams :", common_spam_words)

Mots fréquents dans les hams globalement : ['u', 'im', 'go', 'get', '2', 'come', 'ltgt', 'dont', 'ok', '[explicit]', 'like', 'know', 'call', 'got', 'time']
Mots fréquents dans les spams : ['[entity]', 'free', '2', '[explicit]', 'call[entity]', 'email', 'get', 'click', 'to[entity]', 'txt', 'pleas', 'u', 'call', '4', 'text']


compter le nombre de feature discriminantes.

In [173]:
from rapidfuzz import fuzz

# Fonction générique pour compter un mot clé avec fuzzy matching
def count_keyword_fuzzy(text, keyword, threshold=80):
    count = 0
    for word in text:  # Supposant que le texte est déjà tokenisé
        if fuzz.ratio(word, keyword) >= threshold:
            count += 1
    return count

# Liste des mots-clés à compter
keywords = ['free', 'click', 'claim', 'offer', 'win', 'prize', 'want']

# Ajouter une colonne pour chaque mot-clé avec fuzzy matching
for keyword in keywords:
    column_name = f'w_{keyword}'  # Nom de la colonne, par exemple 'w_free'
    df_all[column_name] = df_all['text'].apply(lambda x: count_keyword_fuzzy(x, keyword))


# Afficher un aperçu des données
print(df_all.head(20))


    label                                               text is_eng  len  \
9       1  [send, final, destin, design, privat, key, 4, ...   True  188   
17      1  [emil, cablefilterz, allow, receiv, channel, o...   True  179   
20      1  [muniz, goven, want, sell, undergroundcd, chec...   True  190   
22      1  [tinydr, 2, 2, gb, 5, x, 2, 25, x, 2, 5, inch,...   True  169   
27      1                 [80, smart, ink, system, [entity]]   True  165   
33      1                                  [visit, [entity]]   True   16   
39      1  [logo, stationeri, design, pleas, visit, site,...   True   53   
48      1  [window, x, p, professi, 0, nal, updat, might,...   True  243   
51      1                               [want, rolex, watch]   True   21   
52      1  [top, rate, onlin, store, hot, new, levitra, l...   True  235   
54      1  [young, teen, take, [explicit], load, face, cl...   True   90   
55      1  [old, realli, fun, right, area, lone, hoys, wl...   True   90   
62      1  [

Enregistrer apres preprosessing

In [174]:
df_all.to_csv("../datasets/preprocessed_df.csv")